# 4.8: Grouping Data & Aggregating Variables


## Import the required libraries


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# Import dataframes
path = r"D:\0 - Data Analytics\4 - Python fundamentals for Data Analysts\Instacart Basket Analisys"

# import dtaframe ords_prods_merge_V2.pkl
ords_prods_merge = pd.read_pickle(
    os.path.join(path, "02 Data", "Prepared Data", "ords_prods_merge_V2.pkl")
)

# create subset of df with first million rows
df = ords_prods_merge[:1000000]


In [2]:
df.shape

(1000000, 19)

In [3]:
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_label,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Regularly busy,Average orders
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Regularly busy,Average orders
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Regularly busy,Average orders
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Regularly busy,Average orders


## Grouping Data


Steps to use groupby():

- Split the data into groups based on some criteria.
- Apply a function to each group separately.
- Combine the results into a dataframe or alternative data structure or create a new column in the current dataframe.


### groupby()

this function will group a given dataframe by a given column.


In [4]:
df.groupby("product_name")

Now I need to aggregate the data to be able to see the results.


### Aggregating Data with agg()


#### Performing Single Aggregation

Calculate the mean of the “order_number” column grouped by the “department_id” column,to quickly compare the average number of orders per user across each Instacart department.


In [5]:
df.groupby("department_id").agg({"order_number": ["mean"]})

,order_number
,mean
department_id,
1,14.800024
2,17.091743
3,17.913544
4,17.893092
5,15.214270
6,15.382135
7,17.694027
8,16.458105


In [6]:
# There are some aggregations that can be conducted without use of the agg() function. For instance, the command above could be replaced with a command that uses the mean() function to achieve the same results:
df.groupby("department_id")["order_number"].mean()

department_id
1     14.800024
2     17.091743
3     17.913544
4     17.893092
5     15.214270
6     15.382135
7     17.694027
8     16.458105
9     15.957363
10    20.091818
11    16.482026
12    15.615061
13    16.484023
14    17.524632
15    15.691875
16    18.014071
17    16.150593
18    19.602850
19    17.631340
20    17.138607
21    21.956893
Name: order_number, dtype: float64

difference in syntax between the two methods:

- When using agg(), put the column you want to aggregate inside the parentheses of the agg() function as an argument.
- When using mean() (or any other standard aggregation function), simply index the column with square brackets, then follow it with the function you want to use after the dot.


#### Dot Notation vs. Square Brackets

df.groupby('department_id').order_number.mean() VS df.groupby("department_id")["order_number"].mean()


Using dot notation results in the same output as the function with brackets. Theoretically, then, you could use dot notation for your commands. However, there are a few reasons why we recommend sticking to square brackets as a general rule:

- Visual Appearance: Square brackets stand out, and using them consistently will help those that may look at your code down the line understand your methods. If you suddenly decided to change to dot notation for certain operations, your colleagues wouldn’t know whether you were indexing or applying a function.
- Default Role: Square brackets have no other role in Python beyond indexing. Dots, however, do (applying pandas functions, looking for certain attributes of a dataframe, etc.). When you start using dot notation for indexing, as well, Python has to work harder to understand what you’re telling it to do, which can make it slower.
  In general, stick to brackets when it comes to indexing!


#### Performing Multiple Aggregation


In [7]:
# Aggregate and produce 'mean' and 'max' for 'order_number' column
df.groupby("department_id").agg({"order_number": ["mean", "min", "max"]})

order_number        
                      mean min max
department_id                     
1                14.800024   1  99
2                17.091743   1  98
3                17.913544   1  99
4                17.893092   1  99
5                15.214270   1  99
6                15.382135   1  99
7                17.694027   1  99
8                16.458105   1  91
9                15.957363   1  99
10               20.091818   1  99
11               16.482026   1  99
12               15.615061   1  99
13               16.484023   1  99
14               17.524632   1  99
15               15.691875   1  99
16               18.014071   1  99
17               16.150593   1  99
18               19.602850   1  99
19               17.631340   1  99
20               17.138607   1  99
21               21.956893   1  97

### Aggregating Data with transform()

Create loyalty flag:

- First transform() to create a column with maximum frequency of 'order_number'
- Then loc() creating a column with 'loyal' or 'not_loyal" flag

Criteria for the flag:

- If the maximum orders the user has made is over 40, then the customer will be labeled a “Loyal customer.”
- If the maximum orders the user has made is over 10 but less than or equal to 40, then the customer will be labeled a “Regular customer.”
- If the maximum orders the user has made is less than or equal to 10, then the customer will be labeled a “New customer.”


##### Transform()

- Split the data into groups based on the “user_id” column.
- Apply the transform() function on the “order_number” column to generate the maximum orders for each user.
- Create a new column, “max_order,” into which you’ll place the results of your aggregation.


In [8]:
ords_prods_merge["max_order"] = ords_prods_merge.groupby(["user_id"])[
    "order_number"
].transform(np.max)

C:\Users\oppre\AppData\Local\Temp\ipykernel_15380\3638217546.py:3: FutureWarning: The provided callable <function max at 0x0000026A47D256C0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ].transform(np.max)


In [9]:
# check the results
pd.options.display.max_rows = None
ords_prods_merge.head(100)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Regularly busy,Average orders,10
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Regularly busy,Average orders,10
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Regularly busy,Average orders,10
5,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10
6,2398795,1,2,3,7,15.0,False,10258,2,0,Pistachios,117,19,3.0,both,Low-range product,Regularly busy,Least busy,Average orders,10
7,2398795,1,2,3,7,15.0,False,12427,3,1,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Least busy,Average orders,10
8,2398795,1,2,3,7,15.0,False,13176,4,0,Bag of Organic Bananas,24,4,10.3,both,Mid-range product,Regularly busy,Least busy,Average orders,10
9,2398795,1,2,3,7,15.0,False,26088,5,1,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Least busy,Average orders,10


##### Deriving Columns with loc()

Create Flag


In [10]:
# create a flag for the loyalty program with loc() function
ords_prods_merge.loc[ords_prods_merge["max_order"] > 40, "loyalty_flag"] = (
    "Loyal customer"
)
ords_prods_merge.loc[
    (ords_prods_merge["max_order"] <= 40) & (ords_prods_merge["max_order"] > 10),
    "loyalty_flag",
] = "Regular customer"
ords_prods_merge.loc[ords_prods_merge["max_order"] <= 10, "loyalty_flag"] = (
    "New customer"
)


In [11]:
# check the results
ords_prods_merge["loyalty_flag"].value_counts(dropna=False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [12]:
# check the head() of specific columns, 'user_id', 'loyalty_flag', 'order_number'
ords_prods_merge[["user_id", "loyalty_flag", "order_number"]].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,1
2,1,New customer,1
3,1,New customer,1
4,1,New customer,1
5,1,New customer,2
6,1,New customer,2
7,1,New customer,2
8,1,New customer,2
9,1,New customer,2


# TASK 4.8


## 1

Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.


Done


## 2

In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.


In [13]:
# Find aggregated mean of order_number grouped by 'department_id'
ords_prods_merge.groupby("department_id").agg({"order_number": ["mean"]})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## 3

Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.


In [14]:
df.groupby("department_id").agg({"order_number": ["mean"]})

,order_number
,mean
department_id,
1,14.800024
2,17.091743
3,17.913544
4,17.893092
5,15.214270
6,15.382135
7,17.694027
8,16.458105


### Analysis of Differences Between Entire Dataframe and Subset Results

The following observations can be made when comparing the mean `order_number` values across `department_id` for the entire dataframe and the subset:

1. **Overall Differences**:

   - The mean `order_number` values in the subset are generally close to those in the full dataset, but some departments show slight variations.
   - Some departments have a higher mean in the subset, while others show a decrease.

2. **Significant Deviations**:

   - `department_id 3` shows the most noticeable increase in mean order number from `17.170395` (full) to `17.913544` (subset), indicating that the subset includes more frequent orders for this department.
   - `department_id 1` has a considerable decrease from `15.457838` to `14.800024`, suggesting that the subset might have filtered out higher order numbers for this department.
   - `department_id 20` shows an increase from `16.473447` to `17.138607`, implying a shift in the subset’s ordering pattern.
   - `department_id 21` has a decrease from `22.902379` to `21.956893`, which, while not drastic, indicates a slightly different distribution in the subset.

3. **Minor Variations**:

   - Some departments, such as `department_id 5` and `department_id 6`, show only minimal changes in their mean order number, suggesting that the subset does not significantly impact these departments.

4. **Potential Causes**:

   - The subset might be filtered based on certain criteria, such as customer segments, product types, or specific time frames, leading to changes in order behavior.
   - Variations in mean values can indicate differences in purchasing trends, where certain departments might have more recurrent customers in the subset.

### Conclusion:

While the subset maintains a similar order distribution across most departments, there are a few notable shifts in mean order numbers. Departments such as `3`, `1`, and `20` exhibit meaningful differences, which could be investigated further to understand customer behavior in different scenarios.


## 4

Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.


Done


## 5

The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.


In [15]:
# Aggregate and produce 'mean','min' and 'max' of 'price' for 'loyalty_flag' column
ords_prods_merge.groupby("loyalty_flag").agg({"prices": ["mean", "min", "max"]})


prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

It looks like that Loyal customers tend to purchase lower costed items(10.4), while newer customers higher priced ones(13.3).
Regular customers fit in the middle (12.5). There's something weird happening because all the custoemrs appear to experience a maximum purchase of 99999.0 this might be an outlier.


## 6

The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

- If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
- If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”


In [16]:
# Create an'avg_product_price' column for each user based on averge price across all their orders
ords_prods_merge["avg_product_price"] = ords_prods_merge.groupby(["user_id"])[
    "prices"
].transform(np.mean)


C:\Users\oppre\AppData\Local\Temp\ipykernel_15380\2907800482.py:4: FutureWarning: The provided callable <function mean at 0x0000026A47D260C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ].transform(np.mean)


In [17]:
# check the result of the new column  and user_id

ords_prods_merge[["user_id", "avg_product_price"]].head(60)

,user_id,avg_product_price
0,1,6.367797
1,1,6.367797
2,1,6.367797
3,1,6.367797
4,1,6.367797
5,1,6.367797
6,1,6.367797
7,1,6.367797
8,1,6.367797
9,1,6.367797


In [18]:
# Create a spending flag for each user based on the average price across all their orders using the following criteria:
# If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
# If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”
ords_prods_merge.loc[ords_prods_merge["avg_product_price"] < 10, "spending_flag"] = (
    "Low spender"
)
ords_prods_merge.loc[ords_prods_merge["avg_product_price"] >= 10, "spending_flag"] = (
    "High spender"
)


In [19]:
# Examining the results
ords_prods_merge["spending_flag"].value_counts(dropna=False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

## 7

In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

- If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
- If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
- If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”


In [20]:
# aggrgate and find the median in the “days_since_prior_order” column using square brackets notation

ords_prods_merge["median_between_orders"] = ords_prods_merge.groupby(["user_id"])[
    "days_since_last_order"
].transform(np.median)


C:\Users\oppre\AppData\Local\Temp\ipykernel_15380\4145879966.py:5: FutureWarning: The provided callable <function median at 0x0000026A47EBCD60> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ].transform(np.median)


In [21]:
# check relevant columns 'user_id', 'days_since_last_order', 'median_between_orders'
ords_prods_merge[["user_id", "days_since_last_order", "median_between_orders"]].head(60)

,user_id,days_since_last_order,median_between_orders
0,1,NaN,20.5
1,1,NaN,20.5
2,1,NaN,20.5
3,1,NaN,20.5
4,1,NaN,20.5
5,1,15.0,20.5
6,1,15.0,20.5
7,1,15.0,20.5
8,1,15.0,20.5
9,1,15.0,20.5


In [22]:
# create order frequency flag for each user based on the median in the “days_since_prior_order” column using the following criteria:
# If the median of the days since a user’s last order is lower than 20, then label them as a “Frequent customer.”
# If the median is between 20 and 30, then label them as an “Regular customer.”
# If the median is higher than 30, then label them as an “Infrequent customer.”
ords_prods_merge.loc[
    ords_prods_merge["median_between_orders"] > 20, "order_frequency_flag"
] = "Frequent customer"

ords_prods_merge.loc[
    (ords_prods_merge["median_between_orders"] > 10)
    & (ords_prods_merge["median_between_orders"] <= 20),
    "order_frequency_flag",
] = "Regular customer"

ords_prods_merge.loc[
    ords_prods_merge["median_between_orders"] <= 10, "order_frequency_flag"
] = "Infrequent customer"


In [23]:
# check and count the results
ords_prods_merge["order_frequency_flag"].value_counts(dropna=False)

order_frequency_flag
Infrequent customer    21559853
Regular customer        7208564
Frequent customer       3636437
NaN                           5
Name: count, dtype: int64

In [24]:
ords_prods_merge.groupby(["order_frequency_flag"]).user_id.nunique()

order_frequency_flag
Frequent customer      59619
Infrequent customer    86596
Regular customer       59993
Name: user_id, dtype: int64

In [25]:
# recheck the results of valuable columns
ords_prods_merge[
    [
        "user_id",
        "days_since_last_order",
        "median_between_orders",
        "order_frequency_flag",
    ]
].head(60)

,user_id,days_since_last_order,median_between_orders,order_frequency_flag
0,1,NaN,20.5,Frequent customer
1,1,NaN,20.5,Frequent customer
2,1,NaN,20.5,Frequent customer
3,1,NaN,20.5,Frequent customer
4,1,NaN,20.5,Frequent customer
5,1,15.0,20.5,Frequent customer
6,1,15.0,20.5,Frequent customer
7,1,15.0,20.5,Frequent customer
8,1,15.0,20.5,Frequent customer
9,1,15.0,20.5,Frequent customer


## 8 Cleaning data removing outliers


In [29]:
# Replace the values over 100 with NaN with np.nan
ords_prods_merge.loc[ords_prods_merge["prices"] > 100, "prices"] = np.nan

## 9

Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.


In [30]:
# Export dataframe
ords_prods_merge.to_pickle(
    os.path.join(path, "02 Data", "Prepared Data", "ords_prods_merge_V3.pkl")
)